# builders

> Generic builders for creating SSE-enabled elements with HTMX

In [ ]:
#| default_exp components.builders

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Dict, Any, Union, Callable
from dataclasses import dataclass, field

## Generic SSE Attribute Helpers

In [ ]:
#| export
def add_sse_attrs(
    element: Any,  # Any element with an attrs dictionary
    endpoint: str,  # SSE endpoint URL
    event_name: str = "message",  # SSE event to listen for
    swap_mode: Optional[str] = None,  # Optional HTMX swap mode **extra_attrs: Additional attributes to add
    **extra_attrs
) -> Any:  # The modified element
    "Add SSE attributes to any element. This function modifies an element in-place by adding the necessary HTMX SSE attributes. It's framework-agnostic and works with any element that has an attrs dictionary."
    if not hasattr(element, 'attrs'):
        element.attrs = {}
    
    element.attrs['hx-ext'] = 'sse'
    element.attrs['sse-connect'] = endpoint
    element.attrs['sse-swap'] = event_name
    
    if swap_mode:
        element.attrs['hx-swap'] = swap_mode
    
    element.attrs.update(extra_attrs)
    return element

In [ ]:
#| export
def add_oob_swap(
    element: Any,  # Any element with an attrs dictionary
    swap_type: str = "innerHTML",  # Type of swap (innerHTML, outerHTML, etc.)
    target_id: Optional[str] = None  # Optional target element ID (uses element's own ID if not specified)
) -> Any:  # The modified element
    "Add out-of-band swap attributes to any element."
    if not hasattr(element, 'attrs'):
        element.attrs = {}
    
    element.attrs['hx-swap-oob'] = swap_type
    
    if target_id and not element.attrs.get('id'):
        element.attrs['id'] = target_id
    
    return element

## SSE Configuration Builder

In [ ]:
#| export
@dataclass
class SSEConfig:
    """
    Configuration for SSE-enabled elements.
    
    This dataclass provides a clean way to configure SSE attributes
    that can be applied to any element.
    """
    endpoint: str
    event_name: str = "message"
    swap_mode: Optional[str] = None
    reconnect_time: Optional[int] = None
    extra_attrs: Dict[str, Any] = field(default_factory=dict)
    
    def apply_to(
        self,
        element: Any  # Element to configure
    ) -> Any:  # The configured element
        """
        Apply this SSE configuration to an element.            
        """
        attrs = self.extra_attrs.copy()
        
        if self.reconnect_time:
            attrs['sse-reconnect-time'] = str(self.reconnect_time)
        
        return add_sse_attrs(
            element,
            self.endpoint,
            self.event_name,
            self.swap_mode,
            **attrs
        )
    
    def to_attrs(
        self
    ) -> Dict[str, str]:  # Dictionary of HTMX SSE attributes
        """
        Convert configuration to attribute dictionary.
        """
        attrs = {
            'hx-ext': 'sse',
            'sse-connect': self.endpoint,
            'sse-swap': self.event_name
        }
        
        if self.swap_mode:
            attrs['hx-swap'] = self.swap_mode
        
        if self.reconnect_time:
            attrs['sse-reconnect-time'] = str(self.reconnect_time)
        
        attrs.update(self.extra_attrs)
        return attrs

## Multi-Update Builder

In [ ]:
#| export
@dataclass
class OOBUpdate:
    """
    Represents an out-of-band update.
    
    This is a generic container for OOB updates that doesn't
    depend on specific element types.
    """
    element: Any
    swap_type: str = "innerHTML"
    target_id: Optional[str] = None
    
    def prepare(
        self
    ) -> Any:  # Element with OOB swap attributes
        """
        Prepare element with OOB attributes.
        """
        return add_oob_swap(self.element, self.swap_type, self.target_id)

In [ ]:
#| export
class MultiUpdateBuilder:
    """
    Builder for creating multiple OOB updates.
    
    This builder is framework-agnostic and works with any elements.
    """
    
    def __init__(
        self,
        container_factory: Optional[Callable] = None  # Optional factory function for creating containers.  Should accept *children as arguments.
    ):
        """
        Initialize the builder.                             
        """
        self.updates: List[OOBUpdate] = []
        self.container_factory = container_factory or (lambda *children: children)
    
    def add(
        self,
        element: Any,  # Element to add
        swap_type: str = "innerHTML",  # Type of swap
        target_id: Optional[str] = None  # Optional target ID
    ) -> 'MultiUpdateBuilder':  # Self for chaining
        """
        Add an update to the builder.            
        """
        self.updates.append(OOBUpdate(element, swap_type, target_id))
        return self
    
    def add_many(
        self,
        updates: List[Union[OOBUpdate, tuple, Any]] # List of updates (OOBUpdate objects, tuples, or elements)
    ) -> 'MultiUpdateBuilder':  # Self for chaining
        """
        Add multiple updates at once.
        """
        for update in updates:
            if isinstance(update, OOBUpdate):
                self.updates.append(update)
            elif isinstance(update, tuple):
                # Assume tuple is (element, swap_type, target_id)
                element = update[0]
                swap_type = update[1] if len(update) > 1 else "innerHTML"
                target_id = update[2] if len(update) > 2 else None
                self.add(element, swap_type, target_id)
            else:
                # Just an element, use defaults
                self.add(update)
        return self
    
    def build(
        self
    ) -> Any:  # Container with all prepared OOB updates
        """
        Build the final multi-update container.            
        """
        prepared = [update.prepare() for update in self.updates]
        return self.container_factory(*prepared)
    
    def clear(
        self
    ) -> 'MultiUpdateBuilder':  # Self for chaining
        """
        Clear all pending updates.
        """
        self.updates = []
        return self
    
    def __len__(
        self
    ) -> int:  # Number of pending updates
        """Get number of pending updates"""
        return len(self.updates)
    
    def __bool__(
        self
    ) -> bool:  # True if there are pending updates, False otherwise
        """Check if there are any updates"""
        return bool(self.updates)

## Script Generation Utilities

In [ ]:
#| export
def generate_sse_cleanup_script(
) -> str:  # JavaScript code as a string
    "Generate JavaScript for proper SSE connection cleanup."
    return """
    (function() {
        function closeSSEConnections() {
            document.querySelectorAll('[sse-connect]').forEach(element => {
                const sseData = element['htmx-internal-data'];
                if (sseData && sseData.sseEventSource) {
                    sseData.sseEventSource.close();
                }
            });
            
            if (typeof htmx !== 'undefined') {
                htmx.findAll('[sse-connect]').forEach(element => {
                    htmx.trigger(element, 'htmx:sseClose');
                });
            }
        }
        
        window.addEventListener('beforeunload', closeSSEConnections);
        window.addEventListener('pagehide', closeSSEConnections);
    })();
    """.strip()

In [ ]:
#| export
def get_htmx_sse_extension_url(
    version: str = "2.2.3"  # Version of the extension
) -> str:  # CDN URL as a string
    "Get the CDN URL for the HTMX SSE extension."
    return f"https://unpkg.com/htmx-ext-sse@{version}/sse.js"

## Helper Functions for Common Patterns

In [ ]:
#| export
def create_sse_attrs(
    endpoint: str,  # SSE endpoint
    event: str = "message",  # Event name to listen for
    swap: Optional[str] = None,  # Optional swap mode **kwargs: Additional attributes
    **kwargs
) -> Dict[str, str]:  # Dictionary of attributes
    "Create a dictionary of SSE attributes. This is a convenience function for creating SSE attributes that can be spread into element constructors."
    attrs = {
        'hx-ext': 'sse',
        'sse-connect': endpoint,
        'sse-swap': event
    }
    
    if swap:
        attrs['hx-swap'] = swap
    
    attrs.update(kwargs)
    return attrs

In [ ]:
#| export
def create_oob_attrs(
    element_id: str,  # ID of target element
    swap_type: str = "innerHTML"  # Type of swap
) -> Dict[str, str]:  # Dictionary of attributes
    "Create a dictionary of OOB swap attributes."
    return {
        'id': element_id,
        'hx-swap-oob': swap_type
    }

## Example Usage

In [ ]:
# Example: Using SSE configuration
def example_sse_config():
    # Create a configuration
    config = SSEConfig(
        endpoint="/sse/notifications",
        event_name="notification",
        swap_mode="afterbegin",
        reconnect_time=5000
    )
    
    print("SSE Configuration:")
    print(f"  Endpoint: {config.endpoint}")
    print(f"  Event: {config.event_name}")
    print(f"  Swap mode: {config.swap_mode}")
    print(f"  Reconnect time: {config.reconnect_time}ms")
    print()
    
    # Convert to attributes
    attrs = config.to_attrs()
    print("As attributes dictionary:")
    for key, value in attrs.items():
        print(f"  {key}: {value}")
    print()
    
    # Apply to a mock element
    class MockElement:
        def __init__(self):
            self.attrs = {}
    
    element = MockElement()
    config.apply_to(element)
    print("Applied to element:")
    print(f"  Element attrs: {element.attrs}")

example_sse_config()

SSE Configuration:
  Endpoint: /sse/notifications
  Event: notification
  Swap mode: afterbegin
  Reconnect time: 5000ms

As attributes dictionary:
  hx-ext: sse
  sse-connect: /sse/notifications
  sse-swap: notification
  hx-swap: afterbegin
  sse-reconnect-time: 5000

Applied to element:
  Element attrs: {'hx-ext': 'sse', 'sse-connect': '/sse/notifications', 'sse-swap': 'notification', 'hx-swap': 'afterbegin', 'sse-reconnect-time': '5000'}


In [ ]:
# Example: Building multi-updates
def example_multi_updates():
    # Mock element class
    class MockElement:
        def __init__(self, content, **attrs):
            self.content = content
            self.attrs = attrs
        
        def __repr__(self):
            return f"MockElement({self.content}, {self.attrs})"
    
    # Create builder with mock container
    builder = MultiUpdateBuilder(
        container_factory=lambda *children: {"container": children}
    )
    
    # Add various updates
    builder.add(
        MockElement("Status: Active", id="status"),
        "innerHTML",
        "status"
    )
    
    builder.add(
        MockElement("42 users", id="count"),
        "outerHTML",
        "user-count"
    )
    
    # Add multiple at once
    builder.add_many([
        (MockElement("Message 1", id="msg1"), "innerHTML"),
        (MockElement("Message 2", id="msg2"), "afterend"),
        MockElement("Message 3", id="msg3")  # Uses defaults
    ])
    
    print(f"Number of updates: {len(builder)}")
    print(f"Has updates: {bool(builder)}")
    print()
    
    # Build and show structure
    result = builder.build()
    print("Built container:")
    print(f"  Type: {type(result)}")
    print(f"  Number of children: {len(result['container'])}")
    
    # Show OOB attributes on children
    print("\nOOB elements:")
    for i, child in enumerate(result['container']):
        print(f"  {i+1}. {child.attrs.get('id', 'no-id')}: "
              f"swap={child.attrs.get('hx-swap-oob', 'none')}")

example_multi_updates()

Number of updates: 5
Has updates: True

Built container:
  Type: <class 'dict'>
  Number of children: 5

OOB elements:
  1. status: swap=innerHTML
  2. count: swap=outerHTML
  3. msg1: swap=innerHTML
  4. msg2: swap=afterend
  5. msg3: swap=innerHTML


In [ ]:
# Example: Using helper functions
def example_helpers():
    # Create SSE attributes
    sse_attrs = create_sse_attrs(
        "/sse/updates",
        event="update",
        swap="innerHTML",
        id="live-updates",
        cls="update-container"
    )
    
    print("SSE attributes:")
    for key, value in sse_attrs.items():
        print(f"  {key}: {value}")
    print()
    
    # Create OOB attributes
    oob_attrs = create_oob_attrs("notification-area", "beforeend")
    
    print("OOB attributes:")
    for key, value in oob_attrs.items():
        print(f"  {key}: {value}")
    print()
    
    # Get extension URL
    url = get_htmx_sse_extension_url("2.1.0")
    print(f"HTMX SSE Extension URL: {url}")
    print()
    
    # Generate cleanup script (show first 200 chars)
    script = generate_sse_cleanup_script()
    print(f"Cleanup script (first 200 chars):")
    print(script[:200] + "...")

example_helpers()

SSE attributes:
  hx-ext: sse
  sse-connect: /sse/updates
  sse-swap: update
  hx-swap: innerHTML
  id: live-updates
  cls: update-container

OOB attributes:
  id: notification-area
  hx-swap-oob: beforeend

HTMX SSE Extension URL: https://unpkg.com/htmx-ext-sse@2.1.0/sse.js

Cleanup script (first 200 chars):
(function() {
        function closeSSEConnections() {
            document.querySelectorAll('[sse-connect]').forEach(element => {
                const sseData = element['htmx-internal-data'];
      ...


## Testing

In [ ]:
# Test: SSE configuration
def test_sse_config():
    config = SSEConfig(
        endpoint="/test",
        event_name="test-event",
        swap_mode="outerHTML",
        reconnect_time=3000,
        extra_attrs={'class': 'test-class'}
    )
    
    attrs = config.to_attrs()
    assert attrs['sse-connect'] == "/test"
    assert attrs['sse-swap'] == "test-event"
    assert attrs['hx-swap'] == "outerHTML"
    assert attrs['sse-reconnect-time'] == "3000"
    assert attrs['class'] == 'test-class'
    
    # Test apply_to
    class MockEl:
        attrs = {}
    
    el = MockEl()
    config.apply_to(el)
    assert el.attrs['sse-connect'] == "/test"
    
    print("✓ SSE configuration tests passed")

test_sse_config()

✓ SSE configuration tests passed


In [ ]:
# Test: Multi-update builder
def test_multi_update_builder():
    class MockEl:
        def __init__(self, id):
            self.attrs = {'id': id}
    
    builder = MultiUpdateBuilder()
    
    # Test adding updates
    builder.add(MockEl('el1'), 'innerHTML', 'target1')
    builder.add(MockEl('el2'), 'outerHTML')
    
    assert len(builder) == 2
    assert bool(builder) is True
    
    # Test add_many
    builder.add_many([
        OOBUpdate(MockEl('el3'), 'beforeend'),
        (MockEl('el4'), 'afterbegin'),
        MockEl('el5')
    ])
    
    assert len(builder) == 5
    
    # Test clear
    builder.clear()
    assert len(builder) == 0
    assert bool(builder) is False
    
    print("✓ Multi-update builder tests passed")

test_multi_update_builder()

✓ Multi-update builder tests passed


In [ ]:
# Test: Helper functions
def test_helpers():
    # Test create_sse_attrs
    attrs = create_sse_attrs("/endpoint", "event", "innerHTML", custom="value")
    assert attrs['sse-connect'] == "/endpoint"
    assert attrs['sse-swap'] == "event"
    assert attrs['hx-swap'] == "innerHTML"
    assert attrs['custom'] == "value"
    
    # Test create_oob_attrs
    oob = create_oob_attrs("my-id", "outerHTML")
    assert oob['id'] == "my-id"
    assert oob['hx-swap-oob'] == "outerHTML"
    
    # Test URL generation
    url = get_htmx_sse_extension_url("2.0.0")
    assert "2.0.0" in url
    assert url.startswith("https://")
    
    # Test script generation
    script = generate_sse_cleanup_script()
    assert "closeSSEConnections" in script
    assert "beforeunload" in script
    
    print("✓ Helper function tests passed")

test_helpers()

✓ Helper function tests passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()